# **Initialization** 

In [1]:
import pandas as pd
import torch

### Control

In [2]:
# Paths
RAW_DATA_FOLDER = 'raw_data'
TARGET_FILE_PATH = 'unprocessed_data'

# Flow Controls
RELOAD_RAW_DATA = False
DATA = 'insomnia'  # Options: 'depression', 'insomnia', 'electrical_circuit'

# System variables
STATE = 42
TEST_SET_FRACTION = 0.20
MISSING_VALUES_THRESHOLD = 0.50
SAMPLES_ELECTRICAL_CIRCUIT = 5000
VERBOSE = True

# **Data Preparation**

### Merge raw data files

In [3]:
from raw_data_loader import load_raw_data

if (RELOAD_RAW_DATA):
    load_raw_data(RAW_DATA_FOLDER, TARGET_FILE_PATH)

### Preprocessing and Split

In [4]:
from preprocessing_depression import clean_and_preprocess_depression_data
from preprocessing_insomnia import clean_and_preprocess_insomnia_data
from preprocessing_electrical_circuit import gen_and_preprocess_ec_data

dataset = pd.read_csv(TARGET_FILE_PATH + '/depression_data.csv')

if DATA == 'depression':
    X_train, X_test, y_train, y_test, y_embed_train, y_embed_test = clean_and_preprocess_depression_data(dataset, RAW_DATA_FOLDER, TEST_SET_FRACTION, STATE, MISSING_VALUES_THRESHOLD)
elif DATA == 'insomnia':
    X_train, X_test, y_train, y_test, y_embed_train, y_embed_test = clean_and_preprocess_insomnia_data(dataset, RAW_DATA_FOLDER, TEST_SET_FRACTION, STATE, MISSING_VALUES_THRESHOLD)
elif DATA == 'electrical_circuit':
    ec_X_train, ec_X_test, ec_y_train, ec_y_test, ec_y_embed_train, ec_y_embed_test = gen_and_preprocess_ec_data(SAMPLES_ELECTRICAL_CIRCUIT, TEST_SET_FRACTION, STATE)
else:
    raise ValueError("Invalid dataset selected")

#TODO Fix issue with the time columns SLQ300/310/320/330 in depression and processing

0    2822
1    1345
Name: count, dtype: int64


c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia.py:696: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["approximate_freq_moderate_LTPA"] = data["PAD790Q"] * data["PAD790U"].map(convert_frequency)
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia.py:697: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["approximate_freq_vigorous_LTPA"] = data["PAD810Q"] * data["PAD810U"].map(convert_frequency)
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia

True
True
True
True


c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia.py:696: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["approximate_freq_moderate_LTPA"] = data["PAD790Q"] * data["PAD790U"].map(convert_frequency)
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia.py:697: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["approximate_freq_vigorous_LTPA"] = data["PAD810Q"] * data["PAD810U"].map(convert_frequency)
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia

True
True
True
True


# **Models**

In [5]:
from baseline_models import train_multitarget_baseline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

### Training Bayesian Models

In [6]:
nb_model = GaussianNB()
y_pred_nb, acc_nb = train_multitarget_baseline(
                            model=nb_model,
                            is_classifier=False,
                            X_train=X_train,
                            X_test=X_test,
                            y_embed_train=y_embed_train,
                            y_embed_test=y_embed_test,
                            verbose=VERBOSE)

ValueError: Input contains NaN

### Training Random Forests

In [7]:
# Parameter
N_ESTIMATORS = 100

In [8]:
rf_model = RandomForestRegressor(n_estimators=N_ESTIMATORS, random_state=STATE, n_jobs=-1)
y_pred_rf, mse_rf = train_multitarget_baseline(
                                    model=rf_model, 
                                    is_classifier=False, 
                                    X_train=X_train, 
                                    X_test=X_test, 
                                    y_embed_train=y_embed_train, 
                                    y_embed_test=y_embed_test,
                                    verbose=VERBOSE)

ValueError: Input contains NaN

### Training Logistic Models

In [ ]:
# Parameters
MAX_ITERATIONS = 1000

In [ ]:
log_model = LogisticRegression(max_iter=MAX_ITERATIONS, class_weight='balanced', random_state=STATE)
y_pred_log, acc_log = train_multitarget_baseline(
                            model=log_model,
                            is_classifier=True,
                            X_train=X_train,
                            X_test=X_test,
                            y_embed_train=y_embed_train,
                            y_embed_test=y_embed_test,
                            verbose=VERBOSE)

### Train Split MLP

In [ ]:
from split_model import train_split_model

torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_split_model( X_train, X_test, y_train, y_test, y_embed_train, y_embed_test )

### Train Joint MLP

In [ ]:
from joint_model import train_joint_model

train_joint_model( X_train, X_test, y_train, y_test, y_embed_train, y_embed_test )